## Efficiency stats
check speedup of refactoring of Dan's code 

In [10]:
import pandas as pd
import glob
from src.data.utils import DATA_DIR, load_raw_data
from src.data.process_team_indicators import index_vars

In [13]:
audl = pd.concat([pd.read_csv(dfteam, index_col=0) for dfteam in glob.glob(f'{DATA_DIR}/raw/*.csv')])
audl.shape

(612095, 58)

In [14]:
def GetPlayers(df_in):
    return df_in['Lineup'].str.split(', ').apply(pd.Series).stack().unique()

def GetGamesPlayed(df_in,plyr):
    return df_in[df_in['Lineup'].str.contains(plyr,na=False)].groupby(['GameID']).ngroups
def GetQuartersPlayed(df_in,plyr):
    return df_in[df_in['Lineup'].str.contains(plyr,na=False)].groupby(['GameID','QuarterID']).ngroups
def GetPointsPlayed(df_in,plyr):
    return df_in[(df_in['Event Type']!='Cessation') & df_in['Lineup'].str.contains(plyr,na=False)].groupby(['GameID','PointID']).ngroups
def GetPossessionsPlayed(df_in,plyr):
    return df_in[(df_in['Event Type']=='Offense')&df_in['Lineup'].str.contains(plyr,na=False)].groupby(['GameID','PointID','PossessionID']).ngroups

def GetPointsWon(df_in,plyr):
    return df_in[df_in['Lineup'].str.contains(plyr,na=False)&(df_in['Event Type']=='Offense')&(df_in.Action=='Goal')].groupby(['GameID','PointID']).ngroups
def GetPointsLost(df_in,plyr):
    return df_in[df_in['Lineup'].str.contains(plyr,na=False)&(df_in['Event Type']=='Defense')&(df_in.Action=='Goal')].groupby(['GameID','PointID']).ngroups

def GetOPointsWon(df_in,plyr):
    return df_in[df_in['Lineup'].str.contains(plyr,na=False)
                 &(df_in['Event Type']=='Offense')
                 &(df_in.Action=='Goal')
                 &(df_in.Line=='O')].groupby(['GameID','PointID']).ngroups
def GetOPointsLost(df_in,plyr):
    return df_in[df_in['Lineup'].str.contains(plyr,na=False)
                 &(df_in['Event Type']=='Defense')
                 &(df_in.Action=='Goal')
                 &(df_in.Line=='O')].groupby(['GameID','PointID']).ngroups

def GetDPointsWon(df_in,plyr):
    return df_in[df_in['Lineup'].str.contains(plyr,na=False)
                 &(df_in['Event Type']=='Offense')
                 &(df_in.Action=='Goal')
                 &(df_in.Line=='D')].groupby(['GameID','PointID']).ngroups
def GetDPointsLost(df_in,plyr):
    return df_in[df_in['Lineup'].str.contains(plyr,na=False)
                 &(df_in['Event Type']=='Defense')
                 &(df_in.Action=='Goal')
                 &(df_in.Line=='D')].groupby(['GameID','PointID']).ngroups

def GetOPointsPlayed(df_in,plyr):
    return df_in[(df_in['Event Type']!='Cessation') & (df_in.Line=='O')&df_in['Lineup'].str.contains(plyr,na=False)].groupby(['GameID','PointID']).ngroups
def GetOPossessionsPlayed(df_in,plyr):
    return df_in[(df_in.Line=='O')&(df_in['Event Type']=='Offense')&df_in['Lineup'].str.contains(plyr,na=False)].groupby(['GameID','PointID','PossessionID']).ngroups
def GetOppOPossessionsPlayed(df_in,plyr):
    return df_in[(df_in.Line=='O')&(df_in['Event Type']=='Defense')&df_in['Lineup'].str.contains(plyr,na=False)].groupby(['GameID','PointID','PossessionID']).ngroups

def GetDPointsPlayed(df_in,plyr):
    return df_in[(df_in['Event Type']!='Cessation') & (df_in.Line=='D')&df_in['Lineup'].str.contains(plyr,na=False)].groupby(['GameID','PointID']).ngroups
def GetDPossessionsPlayed(df_in,plyr):
    return df_in[(df_in.Line=='D')&(df_in['Event Type']=='Offense')&df_in['Lineup'].str.contains(plyr,na=False)].groupby(['GameID','PointID','PossessionID']).ngroups
def GetOppDPossessionsPlayed(df_in,plyr):
    return df_in[(df_in.Line=='D')&(df_in['Event Type']=='Defense')&df_in['Lineup'].str.contains(plyr,na=False)].groupby(['GameID','PointID','PossessionID']).ngroups


def PlayStatsByPlayer(df_in):
    plyrs = GetPlayers(df_in)
    return pd.DataFrame([{'Name': p, 
                          'Games Played': GetGamesPlayed(df_in,p),
                          'Quarters Played': GetQuartersPlayed(df_in,p),
                          'Points Played': GetPointsPlayed(df_in,p),
                          'Possessions Played': GetPossessionsPlayed(df_in,p),
                          'Points Played (Offense)': GetOPointsPlayed(df_in,p),
                          'Possessions Played (Offense)': GetOPossessionsPlayed(df_in,p),
                          'Opp Possessions Played (Offense)': GetOppOPossessionsPlayed(df_in,p),
                          'Points Played (Defense)': GetDPointsPlayed(df_in,p),
                          'Possessions Played (Defense)': GetOPossessionsPlayed(df_in,p),
                          'Opp Possessions Played (Defense)': GetOppDPossessionsPlayed(df_in,p),
                          'Points Won': GetPointsWon(df_in,p),
                          'Points Lost': GetPointsLost(df_in,p),
                          'Points Won (Offense)': GetOPointsWon(df_in,p),
                          'Points Lost (Offense)': GetOPointsLost(df_in,p),
                          'Points Won (Defense)': GetDPointsWon(df_in,p),
                          'Points Lost (Defense)': GetDPointsLost(df_in,p),
                         } for p in plyrs])

In [15]:
%%time
gameplay_stats = audl.groupby(['Teamname','Tournament']).apply(PlayStatsByPlayer).reset_index().rename(columns={'Tournament':'Year'}).drop('level_2',axis=1)

CPU times: user 7min 32s, sys: 1.31 s, total: 7min 33s
Wall time: 7min 34s


In [18]:
def GetPlayers(df_in):
    return df_in['Lineup'].str.split(', ').apply(pd.Series).stack().unique()

def GetGamesPlayed(df_in,plyr):
    return df_in.groupby(['GameID']).ngroups
def GetQuartersPlayed(df_in,plyr):
    return df_in.groupby(['GameID','QuarterID']).ngroups
def GetPointsPlayed(df_in,plyr):
    return df_in[(df_in['Event Type']!='Cessation')].groupby(['GameID','PointID']).ngroups
def GetPossessionsPlayed(df_in,plyr):
    return df_in[(df_in['Event Type']=='Offense')].groupby(['GameID','PointID','PossessionID']).ngroups

def GetPointsWon(df_in,plyr):
    return df_in[(df_in['Event Type']=='Offense')&(df_in.Action=='Goal')].groupby(['GameID','PointID']).ngroups
def GetPointsLost(df_in,plyr):
    return df_in[(df_in['Event Type']=='Defense')&(df_in.Action=='Goal')].groupby(['GameID','PointID']).ngroups

def GetOPointsWon(df_in,plyr):
    return df_in[(df_in['Event Type']=='Offense')
                 &(df_in.Action=='Goal')
                 &(df_in.Line=='O')].groupby(['GameID','PointID']).ngroups
def GetOPointsLost(df_in,plyr):
    return df_in[(df_in['Event Type']=='Defense')
                 &(df_in.Action=='Goal')
                 &(df_in.Line=='O')].groupby(['GameID','PointID']).ngroups

def GetDPointsWon(df_in,plyr):
    return df_in[(df_in['Event Type']=='Offense')
                 &(df_in.Action=='Goal')
                 &(df_in.Line=='D')].groupby(['GameID','PointID']).ngroups
def GetDPointsLost(df_in,plyr):
    return df_in[(df_in['Event Type']=='Defense')
                 &(df_in.Action=='Goal')
                 &(df_in.Line=='D')].groupby(['GameID','PointID']).ngroups

def GetOPointsPlayed(df_in,plyr):
    return df_in[(df_in['Event Type']!='Cessation') & (df_in.Line=='O')].groupby(['GameID','PointID']).ngroups
def GetOPossessionsPlayed(df_in,plyr):
    return df_in[(df_in.Line=='O')&(df_in['Event Type']=='Offense')].groupby(['GameID','PointID','PossessionID']).ngroups
def GetOppOPossessionsPlayed(df_in,plyr):
    return df_in[(df_in.Line=='O')&(df_in['Event Type']=='Defense')].groupby(['GameID','PointID','PossessionID']).ngroups

def GetDPointsPlayed(df_in,plyr):
    return df_in[(df_in['Event Type']!='Cessation') & (df_in.Line=='D')].groupby(['GameID','PointID']).ngroups
def GetDPossessionsPlayed(df_in,plyr):
    return df_in[(df_in.Line=='D')&(df_in['Event Type']=='Offense')].groupby(['GameID','PointID','PossessionID']).ngroups
def GetOppDPossessionsPlayed(df_in,plyr):
    return df_in[(df_in.Line=='D')&(df_in['Event Type']=='Defense')].groupby(['GameID','PointID','PossessionID']).ngroups


def PlayStatsByPlayer(df):
    plyrs = GetPlayers(df)
    dfs = []
    for p in plyrs:
        df_in = df[df['Lineup'].str.contains(p,na=False)]
        df_p = pd.DataFrame({'Name': p, 
                          'Games Played': GetGamesPlayed(df_in,p),
                          'Quarters Played': GetQuartersPlayed(df_in,p),
                          'Points Played': GetPointsPlayed(df_in,p),
                          'Possessions Played': GetPossessionsPlayed(df_in,p),
                          'Points Played (Offense)': GetOPointsPlayed(df_in,p),
                          'Possessions Played (Offense)': GetOPossessionsPlayed(df_in,p),
                          'Opp Possessions Played (Offense)': GetOppOPossessionsPlayed(df_in,p),
                          'Points Played (Defense)': GetDPointsPlayed(df_in,p),
                          'Possessions Played (Defense)': GetOPossessionsPlayed(df_in,p),
                          'Opp Possessions Played (Defense)': GetOppDPossessionsPlayed(df_in,p),
                          'Points Won': GetPointsWon(df_in,p),
                          'Points Lost': GetPointsLost(df_in,p),
                          'Points Won (Offense)': GetOPointsWon(df_in,p),
                          'Points Lost (Offense)': GetOPointsLost(df_in,p),
                          'Points Won (Defense)': GetDPointsWon(df_in,p),
                          'Points Lost (Defense)': GetDPointsLost(df_in,p),
                         }, index=[0])
        # print(len(df_p))
        dfs.append(df_p)
        
    return pd.concat(dfs, ignore_index=True)
        

In [19]:
%%time
gameplay_stats = audl.groupby(['Teamname','Tournament']).apply(PlayStatsByPlayer).reset_index()#.rename(columns={'Tournament':'Year'}).drop('level_2',axis=1)



CPU times: user 4min 34s, sys: 982 ms, total: 4min 35s
Wall time: 4min 35s


In [44]:
def GetPlayers(df):
    return df['Lineup'].str.split(', ').apply(pd.Series).stack().unique()

def GetGamesPlayed(df):
    return df.groupby(['GameID']).ngroups
def GetQuartersPlayed(df):
    return df.groupby(['GameID','QuarterID']).ngroups
def GetPointsPlayed(df):
    return df[(df['Event Type']!='Cessation')].groupby(['GameID','PointID']).ngroups
def GetPossessionsPlayed(df):
    return df[(df['Event Type']=='Offense')].groupby(['GameID','PointID','PossessionID']).ngroups

def GetPointsWon(df):
    return df[(df['Event Type']=='Offense')&(df.Action=='Goal')].groupby(['GameID','PointID']).ngroups
def GetPointsLost(df):
    return df[(df['Event Type']=='Defense')&(df.Action=='Goal')].groupby(['GameID','PointID']).ngroups

def GetOPointsWon(df):
    return df[(df['Event Type']=='Offense')
              &(df.Action=='Goal')
              &(df.Line=='O')].groupby(['GameID','PointID']).ngroups
def GetOPointsLost(df):
    return df[(df['Event Type']=='Defense')
              &(df.Action=='Goal')
              &(df.Line=='O')].groupby(['GameID','PointID']).ngroups

def GetDPointsWon(df):
    return df[(df['Event Type']=='Offense')
              &(df.Action=='Goal')
              &(df.Line=='D')].groupby(['GameID','PointID']).ngroups
def GetDPointsLost(df):
    return df[(df['Event Type']=='Defense')
              &(df.Action=='Goal')
              &(df.Line=='D')].groupby(['GameID','PointID']).ngroups

def GetOPointsPlayed(df):
    return df[(df['Event Type']!='Cessation') & (df.Line=='O')].groupby(['GameID','PointID']).ngroups
def GetOPossessionsPlayed(df):
    return df[(df.Line=='O')&(df['Event Type']=='Offense')].groupby(['GameID','PointID','PossessionID']).ngroups
def GetOppOPossessionsPlayed(df):
    return df[(df.Line=='O')&(df['Event Type']=='Defense')].groupby(['GameID','PointID','PossessionID']).ngroups

def GetDPointsPlayed(df):
    return df[(df['Event Type']!='Cessation') & (df.Line=='D')].groupby(['GameID','PointID']).ngroups
def GetDPossessionsPlayed(df):
    return df[(df.Line=='D')&(df['Event Type']=='Offense')].groupby(['GameID','PointID','PossessionID']).ngroups
def GetOppDPossessionsPlayed(df):
    return df[(df.Line=='D')&(df['Event Type']=='Defense')].groupby(['GameID','PointID','PossessionID']).ngroups


def PlayStatsByPlayer(df):
    plyrs = GetPlayers(df)
    output_dfs = []
    for p in plyrs:
        df_p = df[df['Lineup'].str.contains(p,na=False)]
        df_out = pd.DataFrame({'Name': p,
                             'Games Played': GetGamesPlayed(df_p),
                             'Quarters Played': GetQuartersPlayed(df_p),
                             'Points Played': GetPointsPlayed(df_p),
                             'Possessions Played': GetPossessionsPlayed(df_p),
                             'Points Played (Offense)': GetOPointsPlayed(df_p),
                             'Possessions Played (Offense)': GetOPossessionsPlayed(df_p),
                             'Opp Possessions Played (Offense)': GetOppOPossessionsPlayed(df_p),
                             'Points Played (Defense)': GetDPointsPlayed(df_p),
                             'Possessions Played (Defense)': GetOPossessionsPlayed(df_p),
                             'Opp Possessions Played (Defense)': GetOppDPossessionsPlayed(df_p),
                             'Points Won': GetPointsWon(df_p),
                             'Points Lost': GetPointsLost(df_p),
                             'Points Won (Offense)': GetOPointsWon(df_p),
                             'Points Lost (Offense)': GetOPointsLost(df_p),
                             'Points Won (Defense)': GetDPointsWon(df_p),
                             'Points Lost (Defense)': GetDPointsLost(df_p),
                             }, index=[0])
        # print(len(df_p))
        output_dfs.append(df_out)

    return pd.concat(output_dfs, ignore_index=True)
        

In [8]:
audl = load_raw_data()

/Users/zrankin/github/audl-viz/src/data/utils.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat([pd.read_csv(f'{DATA_DIR}/raw/{f}') for f in files])


In [45]:
df = audl.sample(1000)
df.shape

(1000, 64)

In [46]:
%%time
gameplay_stats = df.groupby(['team', 'year']).apply(PlayStatsByPlayer).reset_index()

CPU times: user 1min 35s, sys: 408 ms, total: 1min 35s
Wall time: 1min 35s


In [35]:
%%time
gameplay_stats = df.groupby(['team', 'year', 'game']).apply(PlayStatsByPlayer).reset_index()

CPU times: user 3min 33s, sys: 447 ms, total: 3min 34s
Wall time: 3min 34s


In [36]:
%%time
gameplay_stats = df.groupby(['team', 'year', 'game', 'opponent']).apply(PlayStatsByPlayer).reset_index()

CPU times: user 3min 54s, sys: 1.08 s, total: 3min 55s
Wall time: 3min 56s


In [38]:
df = audl.sample(10_000)
df.shape

(10000, 64)

In [39]:
%%time
gameplay_stats = df.groupby(['team', 'year']).apply(PlayStatsByPlayer).reset_index()

CPU times: user 2min 16s, sys: 601 ms, total: 2min 17s
Wall time: 2min 17s


In [40]:
%%time
gameplay_stats = df.groupby(['team', 'year', 'game']).apply(PlayStatsByPlayer).reset_index()

CPU times: user 16min 14s, sys: 4.82 s, total: 16min 19s
Wall time: 16min 21s


In [41]:
%%time
gameplay_stats = df.groupby('game').apply(PlayStatsByPlayer).reset_index()

CPU times: user 15min 58s, sys: 3.79 s, total: 16min 1s
Wall time: 16min 3s


In [42]:
def PlayStatsByTeam(df):
    df_out = pd.DataFrame({
                      'Games Played': GetGamesPlayed(df),
                      'Quarters Played': GetQuartersPlayed(df),
                      'Points Played': GetPointsPlayed(df),
                      'Possessions Played': GetPossessionsPlayed(df),
                      'Points Played (Offense)': GetOPointsPlayed(df),
                      'Possessions Played (Offense)': GetOPossessionsPlayed(df),
                      'Opp Possessions Played (Offense)': GetOppOPossessionsPlayed(df),
                      'Points Played (Defense)': GetDPointsPlayed(df),
                      'Possessions Played (Defense)': GetOPossessionsPlayed(df),
                      'Opp Possessions Played (Defense)': GetOppDPossessionsPlayed(df),
                      'Points Won': GetPointsWon(df),
                      'Points Lost': GetPointsLost(df),
                      'Points Won (Offense)': GetOPointsWon(df),
                      'Points Lost (Offense)': GetOPointsLost(df),
                      'Points Won (Defense)': GetDPointsWon(df),
                      'Points Lost (Defense)': GetDPointsLost(df),
                     }, index=[0])
    return df_out
        
    

In [58]:
def calc_gameplay_rates(df):
    df['O conversion rate'] = df['Points Won (Offense)'] / df['Points Played (Offense)']
    df['D conversion rate'] = df['Points Won (Defense)'] / df['Points Played (Defense)']
    return df

In [53]:
gameplay_stats = df.groupby(['team', 'year']).apply(PlayStatsByPlayer)

In [62]:
team_gameplay_stats = df.groupby(['team', 'year']).apply(PlayStatsByTeam)
team_gameplay_stats = calc_gameplay_rates(team_gameplay_stats)
team_conversion_inds = ['O conversion rate', 'D conversion rate']

# TODO 
- How to name the team rates in a way to allow individuals to be compared easily
- Should all of this be done on the fly anyway to allow aggregation at different levels? or do we only want this comparison done at the yearly basis (since players can change teams over years, etc) 
- Should I mock up some plots? 
    - X axis is number of O points played, y is O conversion rate with horizontal as team avg 
    - X axis is number of D points played, y is D conversion rate with horizontal as team avg 
    - (colored by percentage of O points played)
    - (side by side?)

In [63]:
df_test = pd.merge(gameplay_stats, team_gameplay_stats[team_conversion_inds], left_index=True, right_index=True)

In [64]:
df_test.head()

Name  Games Played  Quarters Played  \
team           year                                                      
Atlanta Hustle 2015 0       Brian Moore             5                5   
               2016 0     Dylan Tunnell             5                8   
               2017 0     Dylan Tunnell             3                3   
               2018 0    Will Lindquist             2                2   
Austin Sol     2016 0  Mitchell Bennett             3                3   

                       Points Played  Possessions Played  \
team           year                                        
Atlanta Hustle 2015 0              5                   2   
               2016 0              8                   8   
               2017 0              3                   3   
               2018 0              2                   0   
Austin Sol     2016 0              3                   3   

                       Points Played (Offense)  Possessions Played (Offense)  \
team           year                                                            
Atlanta Hustle 2015 0                        2                             1   
               2016 0                        7                             7   
               2017 0                        3                             3   
               2018 0                        0                             0   
Austin Sol     2016 0                        2                             2   

                       Opp Possessions Played (Offense)  \
team           year                                       
Atlanta Hustle 2015 0                                 1   
               2016 0                                 0   
               2017 0                                 0   
               2018 0                                 0   
Austin Sol     2016 0                                 0   

                       Points Played (Defense)  Possessions Played (Defense)  \
team           year                                                            
Atlanta Hustle 2015 0                        3                             1   
               2016 0                        1                             7   
               2017 0                        0                             3   
               2018 0                        2                             0   
Austin Sol     2016 0                        1                             2   

                       Opp Possessions Played (Defense)  Points Won  \
team           year                                                   
Atlanta Hustle 2015 0                                 2           0   
               2016 0                                 0           1   
               2017 0                                 0           0   
               2018 0                                 2           0   
Austin Sol     2016 0                                 0           0   

                       Points Lost  Points Won (Offense)  \
team           year                                        
Atlanta Hustle 2015 0            1                     0   
               2016 0            0                     1   
               2017 0            0                     0   
               2018 0            0                     0   
Austin Sol     2016 0            0                     0   

                       Points Lost (Offense)  Points Won (Defense)  \
team           year                                                  
Atlanta Hustle 2015 0                      0                     0   
               2016 0                      0                     0   
               2017 0                      0                     0   
               2018 0                      0                     0   
Austin Sol     2016 0                      0                     0   

                       Points Lost (Defense)  O conversion rate  \
team           year                                               
Atlanta Hustle 